In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
df = pd.read_csv("../data/LargeDataSet13_HigherExamsWithClassAndSchoolsAvailable.csv")

In [ ]:
mask = ["date" not in a.lower() for a in df.columns]
df = df[df.columns[mask]]

In [ ]:
col_markers = np.arange(0,len(df.columns))[["Name" in a for a in df.columns]].tolist()
col_markers.append(len(df.columns)+1)

In [ ]:
col_markers

In [ ]:
df.columns

In [ ]:

entries = []
data = []
n_qs = 0
for i, ind in enumerate(col_markers[1:]):
    vals = df.loc[0].values[col_markers[i]+1:ind+1]

    if i == 7:
        maxadd = np.array(df.loc[0].values[col_markers[i]+1:ind], dtype=int).tolist()
        actual_add = [str(a) for a in df.loc[1].values[col_markers[i]+1:ind] ]
    else:
        maxadd = np.array(df.loc[1].values[col_markers[i]+1:ind], dtype=int).tolist()
        actual_add = [str(a) for a in df.loc[0].values[col_markers[i]+1:ind]]
    
    entries.append({
        "maxes":maxadd,
        "qtypes": [str(a) for a in df.loc[4].values[col_markers[i]+1:ind]],
        "difficulty":[int(a) for a in df.loc[3].values[col_markers[i]+1:ind]],
        "actual":actual_add,
        "exam_id":i, 
        #"results":np.nan_to_num(np.array(df.to_numpy()[7:,:][:,col_markers[i]+1:ind], dtype = float)) 
    })
    scores = np.array(df.to_numpy()[7:,:][:,col_markers[i]+1:ind], dtype = float)

    n_qs += scores.shape[1]

        
        
        
    
    np.save( f"../data/processed/scores/Exam_{i}.npy", scores)
    binarised = np.zeros(scores.shape)
    for j in range(len(maxadd)):
        binarised[:,j] = scores[:,j]/maxadd[j]
    np.save( f"../data/processed/binarised/Exam_{i}.npy", binarised)

In [ ]:
with open("../data/Processed_Exams.json", "w") as f:
    json.dump(entries, f)

In [ ]:
import numpy as np

In [ ]:
mask_clean = np.ones(len(scores))
with open("../data/Processed_Exams.json", "r") as f:
    config = json.load(f)

runtot = 0
for i in range(13):
    maxadd = config[i]["maxes"]
    scores = np.load(f"../data/processed/scores/Exam_{i}.npy")
    mask = np.zeros(scores.shape)
    for j, ms in enumerate(maxadd):
        mask[:,j] = (scores[:,j] <=ms) | (np.isnan(scores[:,j]) )
    runtot+=len(maxadd)
    np.save(f"../data/processed/masks/Exam_{i}_Mask.npy",np.all(mask, axis=1))
    mask_clean = mask_clean * np.all(mask, axis=1)
    print("exam", np.sum(np.all(mask, axis=1) & (~np.isnan(scores[:,0]))))

In [ ]:
np.save("../data/outputs/CleanMask.npy", arr = mask_clean)